In [ ]:
import pandas as pd
import numpy as np

#lendo a base de dados
df_vendas = pd.read_excel(r"Bases de Dados/Vendas.xlsx")
display(df_vendas)

In [ ]:
#verificando a base de dados
df_vendas.info()

In [ ]:
#separando o faturamento por loja diario/anaul
faturamento_dia = (
    df_vendas.groupby(["ID Loja", "Data"], as_index=False)["Valor Final"]
    .sum()
    .rename(columns={"Valor Final":"Faturamento Diario"})
    )

df_vendas["Ano"] = df_vendas["Data"].dt.year

faturamento_ano = (
    df_vendas.groupby(["ID Loja", "Ano"], as_index=False)["Valor Final"]
    .sum()
    .rename(columns={"Valor Final":"Faturamento Anual"})
    )

display(faturamento_dia)
display(faturamento_ano.head())

In [ ]:
#separando a variedade de itens vendidos por loja diaria/anual
variedade_itens_dia = (
    df_vendas.groupby(["ID Loja", "Data"], as_index=False)["Produto"]
    .nunique()
    .rename(columns={"Produto":"Variedade Diaria"})
)

variedade_itens_ano = (
    df_vendas.groupby(["ID Loja", "Ano"], as_index=False)["Produto"]
    .nunique()
    .rename(columns={"Produto":"Variedade Anual"})
)

display(variedade_itens_dia)
display(variedade_itens_ano.head())

In [ ]:
#separando o ticket-medio de cada loja dia/ano
ticket_medio_dia = (
    df_vendas.groupby(["ID Loja", "Data"], as_index=False)[["Quantidade", "Valor Final"]]
    .sum()
    .assign(
        Ticket_Medio_Dia=lambda x: (
            x["Valor Final"] / x["Quantidade"].replace(0, pd.NA)
        ).round(2)
    )
    .drop(columns=["Valor Final", "Quantidade"])
    .rename(columns=lambda x: x.replace("_", " "))
)   

ticket_medio_ano = (    
    df_vendas.groupby(["ID Loja", "Ano"], as_index=False)[["Quantidade", "Valor Final"]]
    .sum()
    .assign(
        Ticket_Medio_Ano=lambda x: (
            x["Valor Final"] / x["Quantidade"].replace(0, pd.NA)
        ).round(2)
    )
    .drop(columns=["Valor Final", "Quantidade"])
    .rename(columns=lambda x: x.replace("_", " "))
)

display(ticket_medio_dia)
display(ticket_medio_ano.head())

In [ ]:
#verificando se as metas foram batidas
fat_meta_dia = 1000
fat_meta_ano = 1650000
var_meta_dia = 4
var_meta_ano = 120
ticket_meta = 500 

def verificar_meta(df, valor, meta):
    df = df.copy()
    df["Bateu Meta"] = np.where(df[valor] >= meta, "Sim", "Não")
    return df

lista_dfs = [
    (faturamento_dia, "Faturamento Diario", fat_meta_dia),
    (faturamento_ano, "Faturamento Anual", fat_meta_ano),
    (variedade_itens_dia, "Variedade Diaria", var_meta_dia),
    (variedade_itens_ano, "Variedade Anual", var_meta_ano),
    (ticket_medio_dia, "Ticket Medio Dia", ticket_meta),
    (ticket_medio_ano, "Ticket Medio Ano", ticket_meta)
]

dfs_novos = [
    verificar_meta(df, valor, meta)
    for df, valor, meta in lista_dfs
]

(
    faturamento_dia,
    faturamento_ano,
    variedade_itens_dia,
    variedade_itens_ano,
    ticket_medio_dia,
    ticket_medio_ano
) = dfs_novos


In [ ]:
#criar as onepages
df_email = pd.read_csv(r"Bases de Dados/Emails.csv")
df_lojas = pd.read_csv(r"Bases de Dados/Lojas.csv")

display(df_email)
display(df_lojas)